In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt


CELL 1 — Kaggle Default Environment Cell
 Theory:
This is the default notebook cell automatically provided by Kaggle.
It usually imports basic libraries and checks available data files using os.walk().
Purpose:
To inspect the environment before building agents
To verify that the project can see /kaggle/input/
To ensure that data sources are accessible
Why this matters in Capstone:
Agents systems often depend on external knowledge or stored memory.

In [2]:
import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID  = os.getenv("GOOGLE_CSE_ID")

if GOOGLE_API_KEY is None:
    raise RuntimeError(
        "GOOGLE_API_KEY secret is missing. "
        "Create a Kaggle secret named GOOGLE_API_KEY and attach it to this notebook."
    )

if GOOGLE_CSE_ID is None:
    raise RuntimeError(
        "GOOGLE_CSE_ID secret is missing. "
        "Create a Kaggle secret named GOOGLE_CSE_ID and attach it to this notebook."
    )

print("✅ Loaded Google API keys")

RuntimeError: GOOGLE_API_KEY secret is missing. Create a Kaggle secret named GOOGLE_API_KEY and attach it to this notebook.

CELL 2 — Installation Cell
Theory:
In this cell, we install external libraries needed for our system.
Example installs:
requests → to communicate with Google API (HTTP calls)
→ requests
Why this matters:
Agent systems need tools.
In our architecture:
Agents gather knowledge from search API
Agents summarize and structure information
Agents store long-term notes
To do that, we must install the libraries that perform these operations.
 Installation = enabling the core capabilities of the project.
This cell  also loads sensitive keys from Kaggle Secrets.
You never hard-code API keys inside notebooks.
Instead:
Kaggle → Add-ons → Secrets → create keys
Notebook reads them from environment variables
Why important:
1. Security
Prevents accidental leaks to GitHub or screenshots.
2. Competition Rules
Public notebooks must not expose credentials.
3. Scalability
You can switch keys without touching the code.
In this project:
GOOGLE_API_KEY → authenticates to Google Programmable Search
GOOGLE_CSE_ID → identifies your custom search engine index

In [ ]:
import json
import requests
from pathlib import Path

# Memory Bank (Knowledge Base)
STORE = Path("adk_memory.json")

def load_memory():
    if STORE.exists():
        try:
            return json.loads(STORE.read_text())
        except:
            return []
    return []

def save_memory(data):
    STORE.write_text(json.dumps(data, indent=2))

# Google Ask API Tool (organic clean results)
def google_search(query: str, num=5):
    """
    ASK-style Google Programmable Search.
    Returns only clean title+snippet.
    """
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "q": query,
        "safe": "active",
    }

    data = requests.get(url, params=params).json()
    out = []

    for item in data.get("items", [])[:num]:
        out.append({
            "title": item.get("title", ""),
            "snippet": item.get("snippet", "")
        })
    return out

# Agent 1 — ADK Research Agent
def research_agent(topic):
    print("🔎 ADK Research Agent Searching...\n")
    results = google_search(topic)
    return results

# Agent 2 — ADK Summary Agent
def summary_agent(results):
    print("🧠 ADK Summary Agent Processing...\n")

    lines = []
    for r in results:
        lines.append(f"- {r['title']}: {r['snippet']}")

    text = "\n".join(lines)

    # extract main idea (very simple heuristic)
    return {
        "summary": text[:800],
        "sources": [r["title"] for r in results]
    }

# Agent 3 — ADK Planner Agent
def planner_agent(topic):
    print("📅 ADK Planner Agent Building Plan...\n")

    return {
        "plan": [
            f"Day 1 → Search basics of {topic}",
            f"Day 2 → Review core concepts of {topic}",
            f"Day 3 → Explore examples + practice problems",
            "Day 4 → Attempt quiz/mock test",
            "Day 5 → Revision and write summary notes"
        ]
    }

# Agent 4 — ADK Knowledge Agent (persistent)
def knowledge_agent(topic, data):
    mem = load_memory()
    entry = {
        "topic": topic,
        "data": data
    }
    mem.append(entry)
    save_memory(mem)
    print("ADK Knowledge Agent → stored in memory.\n")

CELL 3 — Multi-Agent ADK System Construction
This is the heart of your project.
Theory:
A multi-agent system is built using the ADK (Agent Development Kit) pattern.
Each agent is a specialized module with a specific responsibility.
1. Memory Layer
We store knowledge in a local JSON file.
Why?
Agents should not “forget”
Knowledge accumulates across sessions
Enables Retrieval Augmented reasoning

This is offline, safe, reusable memory.
 2. Google ASK Search Tool
This is the main “sense organ” of the system.
It:
Asks Google
Retrieves organic title + snippet
This makes your system data-grounded.
Agents do not hallucinate, because they rely on external real data.
 3. Research Agent
This agent collects knowledge.
It
Calls Google
Filters information
Only gathers and shares what was foundno new invention isdone by it.
4. Summary Agent
This agent compresses raw data.
Why?
Google returns scattered info
Snippets need structure
Summaries help planners and users
This is similar to post-processing in enterprise systems.
5. Planner Agent
This agent transforms topics into study plans.
It is responsible for:
Structuring schedules
Dividing content into days
Task-based breakdown
It uses no external models.
Only internal logic + patterns.
6. Knowledge Agent
Final agent that:
Persists insights to memory
Creates a reusable knowledge base
This supports:
Recall
Iterative research
Lifelong agent memory
Why Multi-Agent?
Like a real company:
Role	Responsibility
Research Agent	Find information
Summary Agent	Compress information
Planner Agent	Convert into actionable plan
Knowledge Agent	Store and organize knowledge
This division of duties:
Improves reliability
Makes debugging easier
Makes architecture modular.


In [ ]:
topic = "Operating system deadlock"

# Phase 1: Research
raw_results = research_agent(topic)

# Phase 2: Summarize
processed = summary_agent(raw_results)

# Phase 3: Study Planner
plan = planner_agent(topic)
processed["plan"] = plan["plan"]

# Phase 4: Store knowledge
knowledge_agent(topic, processed)

processed

CELL 4 — Multi-Agent Execution Cell
Theory
This is where the pipeline runs end-to-end:
1. Input Topic
2. Research Agent asks Google
3. Summary Agent compresses results
4. Planner Agent designs task structure
5. Knowledge Agent stores the result
Why this matters:
This cell proves that:
Agents cooperate
Outputs flow from one module to another
The pipeline produces usable, human-friendly knowledge


In [ ]:
mem = load_memory()
for entry in mem:
    print(f"\n🟢 Topic: {entry['topic']}")
    print("➡ Summary:")
    print(entry["data"]["summary"][:300])

CELL 5 — Memory Retrieval Cell
Theory
This cell searches stored knowledge in adk_memory.json.
Why?
Seeing how memory scales matters
Verifies that knowledge is persistent
Allows user to revisit old topics
Agents become:
More intelligent with time
Context aware
Non-volatile
Why This Matters in Capstone
Each cell demonstrates a core requirement of the assignment:
 Tooling
 Multi-agent architecture
 Knowledge retrieval
 Session continuity
 Real external data integration (Google API)
 Memory retention
 No hallucination